In [1]:
# Étape 1: Installation des bibliothèques
!pip install pandas nltk flask
print("--- Cellule 1 terminée: Bibliothèques installées ou déjà à jour. ---")

--- Cellule 1 terminée: Bibliothèques installées ou déjà à jour. ---


In [2]:
# Étape 2: Configuration et Téléchargement Forcé NLTK
import nltk
import os
import shutil # Pour supprimer si besoin

print("Configuration du chemin NLTK...")
# Chemin vers le dossier nltk_data dans votre répertoire personnel
# (Vérifiez/adaptez 'Nicolas' si besoin)
nltk_data_path = os.path.expanduser('~/nltk_data')

# Ajouter ce chemin à NLTK s'il n'y est pas
if nltk_data_path not in nltk.data.path:
    nltk.data.path.append(nltk_data_path)
    print(f"Ajouté {nltk_data_path} au chemin de recherche NLTK.")
else:
    print(f"{nltk_data_path} est déjà dans le chemin de recherche NLTK.")

# Liste des ressources NLTK nécessaires
resources = ['punkt', 'vader_lexicon', 'stopwords']

print("\nVérification/Téléchargement forcé des ressources NLTK...")
for resource in resources:
    resource_path = os.path.join(nltk_data_path, 'corpora' if resource == 'stopwords' else 'tokenizers' if resource == 'punkt' else 'sentiment' if resource == 'vader_lexicon' else '')
    # Pour punkt et vader_lexicon, on cible le sous-dossier spécifique pour la suppression
    if resource == 'punkt':
        specific_path = os.path.join(nltk_data_path, 'tokenizers/punkt')
    elif resource == 'vader_lexicon':
         specific_path = os.path.join(nltk_data_path, 'sentiment/vader_lexicon')
    elif resource == 'stopwords':
         specific_path = os.path.join(nltk_data_path, 'corpora/stopwords')
    else:
         specific_path = None

    # Tenter de supprimer l'ancien dossier si pertinent
    if specific_path and os.path.exists(specific_path):
        print(f"  Tentative de suppression de l'ancien dossier pour '{resource}'...")
        try:
            shutil.rmtree(specific_path)
            print(f"    Ancien dossier '{resource}' supprimé.")
        except Exception as e:
            print(f"    Erreur lors de la suppression de l'ancien dossier '{resource}': {e}")

    # Tenter le téléchargement
    print(f"  Téléchargement/Vérification de '{resource}'...")
    try:
        # On spécifie le dossier de téléchargement pour être sûr
        nltk.download(resource, download_dir=nltk_data_path, force=True, raise_on_error=True)
        print(f"    '{resource}' téléchargé/vérifié avec succès dans {nltk_data_path}.")
    except Exception as e:
        print(f"    ERREUR lors du téléchargement de '{resource}': {e}")
        print(f"    Vérifiez votre connexion internet ou les permissions sur {nltk_data_path}.")

print(f"\nNLTK cherchera les données dans: {nltk.data.path}")
print("--- Cellule 2 terminée: Configuration NLTK et téléchargement des ressources. ---")

Configuration du chemin NLTK...
/Users/Nicolas/nltk_data est déjà dans le chemin de recherche NLTK.

Vérification/Téléchargement forcé des ressources NLTK...
  Tentative de suppression de l'ancien dossier pour 'punkt'...
    Ancien dossier 'punkt' supprimé.
  Téléchargement/Vérification de 'punkt'...


[nltk_data] Downloading package punkt to /Users/Nicolas/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Nicolas/nltk_data...


    'punkt' téléchargé/vérifié avec succès dans /Users/Nicolas/nltk_data.
  Téléchargement/Vérification de 'vader_lexicon'...
    'vader_lexicon' téléchargé/vérifié avec succès dans /Users/Nicolas/nltk_data.
  Tentative de suppression de l'ancien dossier pour 'stopwords'...
    Ancien dossier 'stopwords' supprimé.
  Téléchargement/Vérification de 'stopwords'...
    'stopwords' téléchargé/vérifié avec succès dans /Users/Nicolas/nltk_data.

NLTK cherchera les données dans: ['/Users/Nicolas/nltk_data', '/Users/Nicolas/linkedin_env/nltk_data', '/Users/Nicolas/linkedin_env/share/nltk_data', '/Users/Nicolas/linkedin_env/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']
--- Cellule 2 terminée: Configuration NLTK et téléchargement des ressources. ---


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Nicolas/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# Étape 3: Importations et Initialisations
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer # Importé ici
from collections import Counter
import re
import string
import json

print("Initialisation des composants...")
# Initialiser l'analyseur de sentiment APRES avoir configuré NLTK
try:
    sia = SentimentIntensityAnalyzer()
    print("  Analyseur de sentiment (SentimentIntensityAnalyzer) initialisé.")
except LookupError as e:
    print(f"  ERREUR: Ressource NLTK manquante pour SentimentIntensityAnalyzer ({e}).")
    print("  Vérifiez l'exécution et les messages de la Cellule 2.")
except Exception as e:
     print(f"  Erreur inattendue lors de l'initialisation de SentimentIntensityAnalyzer: {e}")

# Définir les mots vides en français et ajouter des termes spécifiques à l'IA
try:
    stop_words = set(stopwords.words('french'))
    stop_words.update(['lintelligence', 'artificielle', 'lia', 'ia', 'intelligence', 'artificielle']) # Mots vides ajoutés dans app.py
    print("  Liste de mots vides (stopwords) chargée.")
except LookupError as e:
     print(f"  ERREUR: Ressource NLTK manquante pour stopwords ({e}).")
     print("  Vérifiez l'exécution et les messages de la Cellule 2.")
     stop_words = set() # Utiliser un set vide en cas d'erreur
except Exception as e:
     print(f"  Erreur inattendue lors du chargement des stopwords: {e}")
     stop_words = set() # Utiliser un set vide en cas d'erreur


print("--- Cellule 3 terminée: Bibliothèques importées et composants initialisés. ---")

Initialisation des composants...
  Analyseur de sentiment (SentimentIntensityAnalyzer) initialisé.
  Liste de mots vides (stopwords) chargée.
--- Cellule 3 terminée: Bibliothèques importées et composants initialisés. ---


In [4]:
# Étape 4: Définition des Fonctions d'Analyse

print("Définition des fonctions d'analyse...")

# Fonction pour nettoyer le texte
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(f'[{string.punctuation}]', ' ', text)
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

# Fonction pour analyser le sentiment
def analyze_sentiment(text):
    # Vérifie que sia existe et a été initialisé dans la Cellule 3
    if isinstance(text, str) and 'sia' in globals() and isinstance(sia, SentimentIntensityAnalyzer):
        try:
            sentiment = sia.polarity_scores(text)
            return sentiment
        except Exception as e:
            print(f"Erreur pendant l'analyse de sentiment: {e}")
            return {'compound': 0, 'neg': 0, 'neu': 1.0, 'pos': 0}
    # Retour neutre si sia n'est pas prêt ou si le texte est invalide
    return {'compound': 0, 'neg': 0, 'neu': 1.0, 'pos': 0}

# Fonction pour extraire les mots-clés
def extract_keywords(text, top_n=10):
    if not isinstance(text, str) or 'stop_words' not in globals(): # Vérifie que stop_words existe
        return []
    clean = clean_text(text)
    try:
        words = word_tokenize(clean, language='french')
        # Filtrer les mots vides et non alphabétiques
        words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
        return Counter(words).most_common(top_n)
    except LookupError as e:
         print(f"ERREUR: Ressource NLTK manquante pour word_tokenize ({e}). Vérifiez Cellule 2.")
         return [] # Retourner liste vide en cas d'erreur
    except Exception as e:
         print(f"Erreur pendant l'extraction de mots-clés: {e}")
         return []


# Fonction pour calculer le score d'engagement potentiel (inspirée de)
def calculate_engagement_score(text):
    if not isinstance(text, str) or not text: return 0
    sentiment = analyze_sentiment(text)
    longueur = len(text)
    keywords = extract_keywords(text)

    sentiment_positif = sentiment.get('pos', 0) # Utilise .get pour sécurité
    sentiment_negatif = sentiment.get('neg', 0)
    sentiment_factor = (sentiment_positif * 0.7) - (sentiment_negatif * 0.3) # Pondération vue dans app.py

    length_factor = 0 # Inspiré par analyse de et code
    if 500 <= longueur <= 1200: length_factor = 0.25
    elif 100 <= longueur < 500: length_factor = 0.15
    elif 1200 < longueur <= 1800: length_factor = 0.1

    keyword_factor = min(len(keywords), 10) / 20 # Normalisation vue dans app.py

    raw_score_component = sentiment_factor + length_factor + keyword_factor
    score = 40 + raw_score_component * 50 # Mise à l'échelle indicative

    return max(0, min(100, round(score, 2)))


# Fonction pour générer des suggestions d'amélioration (inspirée de)
def generate_suggestions(text, score):
    if not isinstance(text, str): return ["Texte invalide."]
    suggestions = []
    sentiment = analyze_sentiment(text)
    longueur = len(text)
    keywords = extract_keywords(text)

    # Suggestions Sentiment
    if sentiment.get('compound', 0) < 0: suggestions.append("Utilisez un ton plus positif pour augmenter l'engagement.")
    elif sentiment.get('compound', 0) < 0.1 and sentiment.get('pos', 0) < 0.1: suggestions.append("Le ton est très neutre. Envisagez d'ajouter un peu plus d'enthousiasme.")

    # Suggestions Longueur (cf.)
    if longueur < 300: suggestions.append("Votre post est très court. Ajoutez plus de détails/contexte.")
    elif longueur > 1600: suggestions.append("Votre post est assez long. Pensez à le raccourcir ou mieux le structurer.")

    # Suggestions Mots-clés
    if len(keywords) < 4: suggestions.append("Utilisez plus de mots-clés pertinents liés à l'IA.")

    # Suggestions Structure/Engagement (cf.)
    if "?" not in text: suggestions.append("Ajoutez une question ouverte pour encourager l'interaction.")
    if "#" not in text: suggestions.append("N'oubliez pas d'inclure des hashtags pertinents.")
    if not any(emoji in text for emoji in ["😊", "👍", "🚀", "💡", "🤖", "👉", "💪", "🔍", "✅", "⭐", "📊", "📈"]): suggestions.append("Pensez à intégrer quelques émojis pertinents.")

    # Suggestion générale score
    if score < 40: suggestions.append("Score bas. Revoyez ton, longueur, engagement (question, mots-clés).")
    elif score > 85 and not suggestions: suggestions.append("Excellent post, bien optimisé !")
    elif not suggestions and score < 70: suggestions.append("Post correct. Vérifiez clarté et appel à l'action.")

    return suggestions

print("--- Cellule 4 terminée: Fonctions d'analyse définies. ---")

Définition des fonctions d'analyse...
--- Cellule 4 terminée: Fonctions d'analyse définies. ---


In [5]:
# Étape 5: Tester l'Analyseur

print("Lancement du test d'analyse...")

# --- Votre Texte LinkedIn Ici ---
texte_linkedin = """
🚀 Grande nouvelle ! Nous lançons un outil basé sur l'intelligence artificielle pour optimiser vos posts.
Il analyse la pertinence et suggère des améliorations. L'IA change la donne pour le marketing de contenu.
Qu'en pensez-vous ? Comment l'IA impacte-t-elle déjà votre travail ? 🤔
#IA #MarketingDigital #Innovation #ContentMarketing #IntelligenceArtificielle
"""
# ---------------------------------

print(f"--- Analyse du Post ---")
print(f"Texte d'entrée:\n{texte_linkedin}\n")

if texte_linkedin and isinstance(texte_linkedin, str):
    # Exécuter les fonctions définies dans la Cellule 4
    score = calculate_engagement_score(texte_linkedin)
    sentiment = analyze_sentiment(texte_linkedin)
    mots_cles = extract_keywords(texte_linkedin)
    suggestions = generate_suggestions(texte_linkedin, score)

    # Afficher les résultats
    print(f"📊 Score d'Engagement Potentiel: {score}/100")
    print("\n😊 Analyse de Sentiment:")
    print(f"  Positif: {sentiment.get('pos', 0)*100:.2f}%")
    print(f"  Neutre: {sentiment.get('neu', 0)*100:.2f}%")
    print(f"  Négatif: {sentiment.get('neg', 0)*100:.2f}%")
    print(f"  Score Composé VADER: {sentiment.get('compound', 0):.3f}")

    print("\n🔑 Mots-clés Principaux:")
    if mots_cles:
        for mot, compte in mots_cles: print(f"  - {mot} (x{compte})")
    else: print("  - Aucun mot-clé pertinent trouvé (ou erreur NLTK?).")

    print("\n💡 Suggestions d'Amélioration:")
    if suggestions:
        for suggestion in suggestions: print(f"  - {suggestion}")
    else: print("  - Aucune suggestion spécifique.")

    print("\n📄 Statistiques:")
    print(f"  - Longueur: {len(texte_linkedin)} caractères")
    print(f"  - Nombre de mots (approximatif): {len(texte_linkedin.split())}")

else:
    print("Le texte fourni est vide ou invalide. Impossible de l'analyser.")

print("\n--- Cellule 5 terminée: Test d'analyse exécuté. ---")

Lancement du test d'analyse...
--- Analyse du Post ---
Texte d'entrée:

🚀 Grande nouvelle ! Nous lançons un outil basé sur l'intelligence artificielle pour optimiser vos posts.
Il analyse la pertinence et suggère des améliorations. L'IA change la donne pour le marketing de contenu.
Qu'en pensez-vous ? Comment l'IA impacte-t-elle déjà votre travail ? 🤔
#IA #MarketingDigital #Innovation #ContentMarketing #IntelligenceArtificielle


ERREUR: Ressource NLTK manquante pour word_tokenize (
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/french/

  Searched in:
    - '/Users/Nicolas/nltk_data'
    - '/Users/Nicolas/linkedin_env/nltk_data'
    - '/Users/Nicolas/linkedin_env/share/nltk_data'
    - '/Users/Nicolas/linkedin_env/lib/nl

In [6]:
import shutil
import os
import nltk

# Trouver le dossier punkt
punkt_path = os.path.expanduser("~/nltk_data/tokenizers/punkt")

# Supprimer le dossier s'il existe
if os.path.exists(punkt_path):
    shutil.rmtree(punkt_path)
    print("✅ Ancien dossier punkt supprimé.")
else:
    print("ℹ️ Aucun dossier punkt à supprimer.")


✅ Ancien dossier punkt supprimé.


In [7]:
nltk.download('punkt')  # téléchargement standard


[nltk_data] Downloading package punkt to /Users/Nicolas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
# Étape 1: Installation des bibliothèques
!pip install -q pandas nltk flask
print("--- Cellule 1 terminée: Bibliothèques installées ou déjà à jour. ---")


--- Cellule 1 terminée: Bibliothèques installées ou déjà à jour. ---


In [9]:
# Étape 2: Configuration et Téléchargement Forcé NLTK
import nltk
import os
import shutil

print("Configuration du chemin NLTK...")
nltk_data_path = os.path.expanduser('~/nltk_data')

if nltk_data_path not in nltk.data.path:
    nltk.data.path.append(nltk_data_path)
    print(f"Ajouté {nltk_data_path} au chemin de recherche NLTK.")
else:
    print(f"{nltk_data_path} est déjà dans le chemin de recherche NLTK.")

# Supprimer punkt corrompu si besoin
punkt_path = os.path.join(nltk_data_path, 'tokenizers/punkt')
if os.path.exists(punkt_path):
    shutil.rmtree(punkt_path)
    print("✅ Ancien dossier 'punkt' supprimé.")

# Re-télécharger les ressources nécessaires
resources = ['punkt', 'stopwords', 'vader_lexicon']
for res in resources:
    nltk.download(res, download_dir=nltk_data_path, force=True, raise_on_error=True)
print(f"\nNLTK cherchera les données dans: {nltk.data.path}")
print("--- Cellule 2 terminée: Configuration NLTK OK. ---")


Configuration du chemin NLTK...
/Users/Nicolas/nltk_data est déjà dans le chemin de recherche NLTK.


[nltk_data] Downloading package punkt to /Users/Nicolas/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Nicolas/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Nicolas/nltk_data...



NLTK cherchera les données dans: ['/Users/Nicolas/nltk_data', '/Users/Nicolas/linkedin_env/nltk_data', '/Users/Nicolas/linkedin_env/share/nltk_data', '/Users/Nicolas/linkedin_env/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']
--- Cellule 2 terminée: Configuration NLTK OK. ---


In [10]:
# Étape 3: Importations et Initialisations
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from collections import Counter
import re, string, json

print("Initialisation des composants...")

try:
    sia = SentimentIntensityAnalyzer()
    print("  ✅ Analyseur de sentiment initialisé.")
except Exception as e:
    print(f"  ❌ Erreur d'initialisation de SentimentIntensityAnalyzer: {e}")

try:
    stop_words = set(stopwords.words('french'))
    stop_words.update({'lintelligence', 'artificielle', 'lia', 'ia', 'intelligence'})
    print("  ✅ Stopwords français chargés.")
except Exception as e:
    stop_words = set()
    print(f"  ❌ Erreur chargement stopwords: {e}")

print("--- Cellule 3 terminée ---")


Initialisation des composants...
  ✅ Analyseur de sentiment initialisé.
  ✅ Stopwords français chargés.
--- Cellule 3 terminée ---


In [11]:
# Étape 4: Fonctions d'Analyse

def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(f"[{string.punctuation}]", " ", text)
        text = re.sub(r"\d+", "", text)
        text = re.sub(r"\s+", " ", text).strip()
        return text
    return ""

def analyze_sentiment(text):
    if isinstance(text, str) and 'sia' in globals():
        try:
            return sia.polarity_scores(text)
        except:
            return {'compound': 0, 'neg': 0, 'neu': 1.0, 'pos': 0}
    return {'compound': 0, 'neg': 0, 'neu': 1.0, 'pos': 0}

def extract_keywords(text, top_n=10):
    if not isinstance(text, str) or 'stop_words' not in globals():
        return []
    clean = clean_text(text)
    try:
        words = word_tokenize(clean)  # ✅ ne PAS mettre language='french'
        words = [w.lower() for w in words if w.isalpha() and w.lower() not in stop_words]
        return Counter(words).most_common(top_n)
    except:
        return []

def calculate_engagement_score(text):
    if not isinstance(text, str): return 0
    sentiment = analyze_sentiment(text)
    longueur = len(text)
    keywords = extract_keywords(text)

    sentiment_score = sentiment.get('pos', 0)*0.7 - sentiment.get('neg', 0)*0.3
    length_score = 0.25 if 500 <= longueur <= 1200 else 0.15 if longueur < 500 else 0.1 if longueur <= 1800 else 0
    keyword_score = min(len(keywords), 10) / 20

    score = 40 + (sentiment_score + length_score + keyword_score) * 50
    return max(0, min(100, round(score, 2)))

def generate_suggestions(text, score):
    if not isinstance(text, str): return ["Texte invalide."]
    s = []
    sentiment = analyze_sentiment(text)
    longueur = len(text)
    keywords = extract_keywords(text)

    if sentiment['compound'] < 0: s.append("Utilisez un ton plus positif.")
    elif sentiment['compound'] < 0.1 and sentiment['pos'] < 0.1: s.append("Ajoutez un peu plus d'enthousiasme.")

    if longueur < 300: s.append("Ajoutez plus de détails.")
    elif longueur > 1600: s.append("Texte long, structurez ou raccourcissez.")

    if len(keywords) < 4: s.append("Ajoutez des mots-clés liés à l'IA.")
    if "?" not in text: s.append("Ajoutez une question pour favoriser l'engagement.")
    if "#" not in text: s.append("Ajoutez des hashtags.")
    if not any(e in text for e in ["🚀", "🤖", "💡", "👉", "😊", "✅", "📊", "📈"]): s.append("Ajoutez quelques émojis pour le dynamisme.")

    if score < 40: s.append("Score bas. Revoir ton, clarté, engagement.")
    elif score > 85 and not s: s.append("Excellent post, rien à redire !")
    elif not s: s.append("Post correct. Ajoutez peut-être un appel à l'action.")

    return s

print("--- Cellule 4 terminée: Fonctions définies. ---")


--- Cellule 4 terminée: Fonctions définies. ---


In [22]:
# Étape 5: Test de l'analyseur
texte_linkedin = """
🚀 Grande nouvelle ! laéjà votre travail ? 🤔
#IA #MarketingDigital je n'en vreiens pas ::::#Innovation #ContentMarketing #IntelligenceArtificielle
"""

print("--- Analyse du Post ---\n")
print(texte_linkedin)

score = calculate_engagement_score(texte_linkedin)
sentiment = analyze_sentiment(texte_linkedin)
keywords = extract_keywords(texte_linkedin)
suggestions = generate_suggestions(texte_linkedin, score)

print(f"\n📊 Score d'Engagement: {score}/100")
print(f"\n😊 Sentiment : Positif {sentiment['pos']*100:.1f}%, Neutre {sentiment['neu']*100:.1f}%, Négatif {sentiment['neg']*100:.1f}%")
print(f"  Score VADER Composé : {sentiment['compound']:.2f}")

print("\n🔑 Mots-clés :")
for mot, n in keywords: print(f"  - {mot} (x{n})")

print("\n💡 Suggestions :")
for s in suggestions: print(f"  - {s}")


--- Analyse du Post ---


🚀 Grande nouvelle ! laéjà votre travail ? 🤔
#IA #MarketingDigital je n'en vreiens pas ::::#Innovation #ContentMarketing #IntelligenceArtificielle


📊 Score d'Engagement: 47.5/100

😊 Sentiment : Positif 0.0%, Neutre 100.0%, Négatif 0.0%
  Score VADER Composé : 0.00

🔑 Mots-clés :

💡 Suggestions :
  - Ajoutez un peu plus d'enthousiasme.
  - Ajoutez plus de détails.
  - Ajoutez des mots-clés liés à l'IA.
